In [48]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [49]:
words=open('../names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [50]:
chars=sorted(list(set(''.join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}
print(stoi.items())

dict_items([('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5), ('f', 6), ('g', 7), ('h', 8), ('i', 9), ('j', 10), ('k', 11), ('l', 12), ('m', 13), ('n', 14), ('o', 15), ('p', 16), ('q', 17), ('r', 18), ('s', 19), ('t', 20), ('u', 21), ('v', 22), ('w', 23), ('x', 24), ('y', 25), ('z', 26), ('.', 0)])


In [51]:
#context_length
block_size=3
X,Y=[],[]
for w in words:#[:5]:
    context=[0]*block_size
    for ch in w+'.':
        ix=stoi[ch]
        X.append(context.copy())
        Y.append(ix)
        context=context[1:]+[ix]

X=torch.tensor(X)
Y=torch.tensor(Y)
print(X)

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])


In [52]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.emb=torch.nn.Embedding(len(stoi), 16)
        self.fc=torch.nn.Linear(16*block_size, len(stoi))
    def forward(self, x):
        x=self.emb(x)
        x=x.view(x.size(0), -1)
        x=self.fc(x)
        return x
    
model=MLP()
model(X)

tensor([[-3.5356e-01,  2.6479e-01, -4.3044e-01,  ...,  6.4154e-01,
         -9.4623e-01, -3.4054e-01],
        [-1.3720e-01,  6.1982e-01,  1.2998e-01,  ..., -3.5719e-01,
         -1.0261e+00, -1.1294e+00],
        [ 4.4675e-01,  1.2102e+00,  3.0289e-01,  ...,  6.5417e-03,
         -9.4869e-01, -1.3739e+00],
        ...,
        [ 6.3385e-01, -1.7722e-01,  1.1700e-01,  ...,  2.9384e-01,
         -2.5005e-02, -8.4717e-01],
        [-2.9658e-01,  2.0872e-01,  1.1213e+00,  ...,  5.9208e-01,
         -5.0538e-01, -2.4788e-01],
        [ 3.4116e-04, -2.8407e-01,  5.5483e-01,  ...,  5.5035e-01,
          2.5233e-02, -7.4516e-01]], grad_fn=<AddmmBackward0>)

In [53]:
C=torch.randn((27,2))
#F.one_hot(torch.tensor(5),27).float()@C

C[X]


tensor([[[ 0.1429,  0.8882],
         [ 0.1429,  0.8882],
         [ 0.1429,  0.8882]],

        [[ 0.1429,  0.8882],
         [ 0.1429,  0.8882],
         [-1.7787,  1.4682]],

        [[ 0.1429,  0.8882],
         [-1.7787,  1.4682],
         [-0.6947,  1.0899]],

        ...,

        [[-0.0169,  0.3239],
         [-0.0169,  0.3239],
         [ 0.4695,  1.6270]],

        [[-0.0169,  0.3239],
         [ 0.4695,  1.6270],
         [-0.0169,  0.3239]],

        [[ 0.4695,  1.6270],
         [-0.0169,  0.3239],
         [ 0.7580,  0.8357]]])

In [54]:
emb=C[X]
print(emb.shape)
W1=torch.randn((6,100))
b1=torch.randn((100,))

#torch.cat(torch.unbind(emb,1),1)
a=torch.arange(18)
a.view(9,2)

a.storage()


torch.Size([228146, 3, 2])


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [55]:
g=torch.Generator().manual_seed(42)
C=torch.randn((27,2), generator=g)
W1=torch.randn((6,100), generator=g)
b1=torch.randn((100,), generator=g)
W2=torch.randn((100,27), generator=g)
b2=torch.randn((27,), generator=g)
parameters=[C,W1,b1,W2,b2]

In [56]:
sum(p.numel() for p in parameters)

3481

In [57]:
#Forward pass
emb=C[X]
h=torch.tanh(emb.view(-1,6)@W1+b1)
logits=h@W2+b2
counts=logits.exp()
prob=counts/counts.sum(-1, keepdims=True)
loss=-prob[torch.arange(len(Y)),Y].log().mean()
loss


tensor(16.7265)

In [58]:
for p in parameters:
    p.requires_grad=True

In [61]:
for _ in range(1000):
    ix= torch.randint(0,len(X), (32,))
    emb=C[X[ix]]
    h=torch.tanh(emb.view(-1,6)@W1+b1)
    logits=h@W2+b2
    #counts=logits.exp()
    #prob=counts/counts.sum(-1, keepdims=True)
    #loss=-prob[torch.arange(len(Y)),Y].log().mean()
    import torch.nn.functional as F
    loss=F.cross_entropy(logits,Y[ix])
    loss

    for p in parameters:
        p.grad=None
    
    loss.backward()

    for p in parameters:
        p.data+=-p.grad*0.1
    print(loss.item())

2.600144863128662
2.756793737411499
2.8038859367370605
2.356370210647583
2.6970160007476807
2.4960522651672363
2.547781467437744
2.25125789642334
2.2774441242218018
3.069180965423584
2.576930522918701
2.6800737380981445
2.7223591804504395
3.086751699447632
2.272740125656128
2.801710605621338
2.297560930252075
2.7326765060424805
2.7283554077148438
2.5169055461883545
2.602668046951294
2.7036867141723633
2.958566904067993
2.9651026725769043
2.2940587997436523
2.324488401412964
2.7450485229492188
2.231623411178589
2.5782954692840576
2.413334369659424
2.3432302474975586
2.3381004333496094
2.6005187034606934
2.5995235443115234
2.6207785606384277
2.713402032852173
3.001650094985962
2.453256845474243
2.47052001953125
3.0392255783081055
2.6771960258483887
2.673753499984741
2.7411041259765625
2.9635708332061768
2.845703125
2.7082037925720215
2.9104063510894775
2.6684489250183105
2.802851915359497
2.366225242614746
2.482975959777832
3.4207842350006104
2.7579588890075684
2.4408340454101562
2.84603